In [64]:
# import modules
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

# build spark session and spark context
spark = SparkSession.builder \
        .master("local[4]") \
        .appName("hotel") \
        .getOrCreate()
sc = spark.sparkContext

df = spark.read.csv('hotel_bookings.csv',  inferSchema=True, header = True)

In [2]:
from pyspark.sql.functions import col

In [3]:
# replace the strings "NULL" and "NA" with null value
df_withNull = df.replace('NULL', None).replace('NA', None)

# replace null values in 'children' to 0 since there are only 4
df2 = df_withNull.fillna({'children':0})

# replace 'children' datatype to int
df2 = df2.withColumn('children', df2['children'].cast("Int"))


# drop 'company' and 'agent' due to high null count
df2 = df2.drop('agent', 'company','country', 'arrival_date_week_number')

In [4]:
#numerically encode all columns of type string

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, OneHotEncoderModel, StringIndexer

col_string=['hotel', 'arrival_date_month','meal','market_segment','distribution_channel','reserved_room_type','assigned_room_type','deposit_type','customer_type','reservation_status','reservation_status_date']
col_num=[x+"_NUMERIC" for x in col_string]

#add other columns for encoding
col_num=col_num+['arrival_date_year','arrival_date_day_of_month']

col_oh=[x+"_oh" for x in col_string]

col_oh=col_oh+['arrival_date_year_oh','arrival_date_day_of_month_oh']

In [5]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_NUMERIC").fit(df2) for column in col_string]
pipeline = Pipeline(stages=indexers)
df_indexed = pipeline.fit(df2).transform(df2)

In [6]:
#one-hot encode all columns in col_num
ohe = OneHotEncoder(dropLast=False)
ohe.setInputCols(col_num)
ohe.setOutputCols(col_oh)
model = ohe.fit(df_indexed)

df_casted=model.transform(df_indexed)

In [7]:
df_encoded=df_casted.drop(*col_string)
df_encoded=df_encoded.drop(*col_num)

In [8]:
columns_ = df_encoded.columns
columns_

['is_canceled',
 'lead_time',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'booking_changes',
 'days_in_waiting_list',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'distribution_channel_oh',
 'customer_type_oh',
 'reservation_status_date_oh',
 'arrival_date_day_of_month_oh',
 'reservation_status_oh',
 'market_segment_oh',
 'reserved_room_type_oh',
 'assigned_room_type_oh',
 'meal_oh',
 'hotel_oh',
 'deposit_type_oh',
 'arrival_date_year_oh',
 'arrival_date_month_oh']

In [13]:
#from pyspark.mllib.feature import Normalizer
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import Normalizer
from pyspark.mllib.linalg import Vectors

df_encoded=df_casted.drop(*col_string)
df_encoded=df_encoded.drop(*col_num)

#columns of all vector type data that are going to be normalied 
columns_ = ['distribution_channel_oh',
 'customer_type_oh',
 'reservation_status_date_oh',
 'arrival_date_day_of_month_oh',
 'reservation_status_oh',
 'market_segment_oh',
 'reserved_room_type_oh',
 'assigned_room_type_oh',
 'meal_oh',
 'hotel_oh',
 'deposit_type_oh',
 'arrival_date_year_oh',
 'arrival_date_month_oh']

# old normalizer input: normalizer = Normalizer(inputCol=column, outputCol=column+"_NORM", p=1.0) 
#normalizing all vector type data with l^1 norm
for column in columns_:
    normalizer = Normalizer(inputCol=column, outputCol=column+"_NORM", p=1.0) 
    l1NormData = normalizer.transform(df_encoded)
    df_encoded = l1NormData
l1NormData = df_encoded
print("Normalized using L^1 norm")

l1NormData = l1NormData.drop(*columns_)
#l1NormData.show()

Normalized using L^1 norm


In [38]:
l1NormData.columns

['is_canceled',
 'lead_time',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'booking_changes',
 'days_in_waiting_list',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'distribution_channel_oh_NORM',
 'customer_type_oh_NORM',
 'reservation_status_date_oh_NORM',
 'arrival_date_day_of_month_oh_NORM',
 'reservation_status_oh_NORM',
 'market_segment_oh_NORM',
 'reserved_room_type_oh_NORM',
 'assigned_room_type_oh_NORM',
 'meal_oh_NORM',
 'hotel_oh_NORM',
 'deposit_type_oh_NORM',
 'arrival_date_year_oh_NORM',
 'arrival_date_month_oh_NORM']

In [11]:
# using less columns as a test, will remove later
test_columns = ['is_canceled', 'lead_time', 'stays_in_weekend_nights']

In [50]:
# compare this to Classification of Wisconsin Breast Cancer Database assignment
# load modules
import pandas as pd
import pyspark.sql.functions as F
import pyspark.mllib.regression as reg
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.feature import VectorAssembler

# combine all feature columns (non-label columns) into one vector named "feature"
assembler = VectorAssembler(
    inputCols=test_columns,  # testing using 2 features
    outputCol="features")
transformed = assembler.transform(l1NormData)

dataRdd = transformed.select(col("is_canceled").alias("label"), col("features")).rdd.map(tuple)

In [51]:
dataRdd.take(3)

[(0, SparseVector(45, {2: 1.0, 38: 1.0})),
 (0, SparseVector(45, {2: 1.0, 38: 1.0})),
 (0, SparseVector(45, {2: 1.0, 38: 1.0}))]

In [52]:
lp = dataRdd.map(lambda row: Vectors.dense(row[1].toArray()))

In [56]:
lp = dataRdd.map(lambda row: Vectors.dense(row[1].toArray())) \
            .map(lambda row: LabeledPoint(row[0], row[1].toArray()))
data
# .toArray() is needed because some of the variables are vectors, but some 
# variables are ints so i need to find a way around that too

In [65]:
# this fails but the key point of this error is "AttributeError: 'numpy.float64' object has no attribute 'toArray'"
lp.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 47.0 failed 1 times, most recent failure: Lost task 0.0 in stage 47.0 (TID 89, jupyter-mpy, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/opt/conda/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/conda/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/conda/lib/python3.7/site-packages/pyspark/rdd.py", line 1440, in takeUpToNumLeft
    yield next(iterator)
  File "/opt/conda/lib/python3.7/site-p

In [ ]:
# 70/30 test train split
# seed = 42
# train, test = lp_normdata.randomSplit([0.7, 0.3], seed)